In [1]:
import tensorflow as tf
import os
import numpy as np
from model import *
from utility import *

model_name = "AC-DRAGAN_for_DTD"

In [2]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.g_bn0 = BatchNormalization(name = 'g_bn0')
        self.g_bn1 = BatchNormalization(name = 'g_bn1')
        self.g_bn2 = BatchNormalization(name = 'g_bn2')
        self.g_bn3 = BatchNormalization(name = 'g_bn3')

    def __call__(self, z):
        with tf.variable_scope('g', reuse=self.reuse):

            fc0 = full_connection_layer(z, 512*6*6, name="fc0")
            fc0 = tf.reshape(fc0, [-1, 6, 6, 512])

            batch_size = tf.shape(fc0)[0]
            deconv0 = deconv2d_layer(fc0, [batch_size, 12, 12, 256], kernel_size=5, name="deconv0")
            deconv0 = self.g_bn0(deconv0)
            deconv0 = lrelu(deconv0, leak=0.3)

            deconv1 = deconv2d_layer(deconv0, [batch_size, 24, 24, 128], kernel_size=5, name="deconv1")
            deconv1 = self.g_bn1(deconv1)
            deconv1 = lrelu(deconv1, leak=0.3)

            deconv2 = deconv2d_layer(deconv1, [batch_size, 48, 48, 64], kernel_size=5, name="deconv2")
            deconv2 = self.g_bn2(deconv2)
            deconv2 = lrelu(deconv2, leak=0.3)

            deconv3 = deconv2d_layer(deconv2, [batch_size, 96, 96, 3], kernel_size=5, name="deconv3")

            output = tf.nn.sigmoid(deconv3)
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [3]:
class Discriminator:
    def __init__(self, cat_size):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        self.d_bn2 = BatchNormalization(name="d_bn2")
        self.d_bn3 = BatchNormalization(name="d_bn3")
        self.d_bn4 = BatchNormalization(name="d_bn4")
        
        self.cat_size = cat_size

    def __call__(self, x):
        with tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 96, 96, 3])

            conv1 = conv2d_layer(x, 64, kernel_size=5, name="d_conv0")
            conv1 = self.d_bn0(conv1)
            conv1 = lrelu(conv1, leak=0.3)
            
            conv2 = conv2d_layer(conv1, 128, kernel_size=5, name="d_conv1")
            conv2 = self.d_bn1(conv2)
            conv2 = lrelu(conv2, leak=0.3)

            conv3 = conv2d_layer(conv2, 256, kernel_size=5, name="d_conv2")
            conv3 = self.d_bn2(conv3)
            conv3 = lrelu(conv3, leak=0.3)

            conv4 = conv2d_layer(conv3, 512, kernel_size=5, name="d_conv3")
            conv4 = self.d_bn3(conv4)
            conv4 = lrelu(conv4, leak=0.3)
            
            disc = full_connection_layer(conv4, 1, name="disc")
            disc = tf.nn.sigmoid(disc)

            aux = full_connection_layer(conv4, self.cat_size, name="aux")

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return disc, aux

In [4]:
class GAN:
    def __init__(self):
        self.batch_size = 64
        self.img_size = 96
        self.rand_size = 50
        self.cat_size = 47
        self.z_size = self.rand_size + self.cat_size
        
        self.epochs = 50000
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[]}

        # unrolled counts
        self.steps = 5

        #self.dataset = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
        #self.dataset = (self.dataset/255)# - 0.5

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 3])
        self.Y_tr = tf.placeholder(tf.float32, shape=[None, self.cat_size])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        self.X_per = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 3])

        self.g = Generator()
        self.d = Discriminator(self.cat_size)
        self.Xg = self.g(self.z)
        self.dtd = DTD()

    def loss(self):
        disc_tr, aux_tr = self.d(self.X_tr)
        disc_gen, aux_gen = self.d(self.Xg)
        
        lambda_adv = 34
        lambda_gp = 0.5

        TINY = 1e-8
        loss_g = lambda_adv*tf.reduce_mean(1 - tf.log(disc_gen + TINY))

        diff = self.X_per - self.X_tr
        #print(g_outputs.shape[0])
        alpha = tf.random_uniform(shape=[self.batch_size,1,1,1], minval=0., maxval=1.)
        interpolates = self.X_tr + (alpha*diff)
        disc_interplates, _ = self.d(interpolates)
        gradients = tf.gradients(disc_interplates, [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)


        TINY = 1e-8
        loss_d = -lambda_adv*tf.reduce_mean(tf.log(disc_tr + TINY) + tf.log(1. - disc_gen + TINY)) + lambda_gp*gradient_penalty

        loss_c_tr = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=aux_tr, labels=self.Y_tr))
        loss_c_gen = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=aux_gen, labels=self.Y_tr))
        loss_c = (loss_c_tr + loss_c_gen)

        loss_g += loss_c
        loss_d += loss_d
        return loss_g, loss_d

    def train(self):
        # Optimizer
        d_lr = 2e-4
        d_beta1 = 0.5
        g_lr = 2e-4
        g_beta1 = 0.5

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "0"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.z_size])

            for epoch in range(self.epochs):
                for step in range(self.steps):
                    # extract images for training
                    #rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                    #X_mb, Y_mb = self.dataset[rand_index, :].astype(np.float32)
                    X_mb, Y_mb = self.dtd.extract(self.batch_size, self.img_size)
                    X_mb = np.reshape(X_mb, [-1, 96, 96, 3])
                    X_mb_per = X_mb + 0.5*np.std(X_mb)*np.random.random(X_mb.shape)

                    rand = np.random.uniform(-1, 1, size=[self.batch_size, self.rand_size])
                    #print(rand.shape)
                    #print(Y_mb.shape)
                    z = np.hstack((rand, Y_mb))
                    #print(z.shape)

                    # train Discriminator
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                        self.X_tr: X_mb,
                        self.z:z,
                        self.Y_tr: Y_mb,
                        self.X_per: X_mb_per,
                    })

                    # train Generator
                    _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                        self.X_tr: X_mb,
                        self.z:z,
                        self.Y_tr: Y_mb,
                        self.X_per: X_mb_per,
                    })

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                # print epoch
                if epoch % 50 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss: {2} '.format(epoch, d_loss_value, g_loss_value))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    img = (img+0.5)
                    save_imgs(model_name, img, name=str(epoch))

                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [5]:
gan = GAN()
gan.train()

init DTD
datasize of texture:5641


/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:80: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


epoch:0, d_loss:1.005228877067566, g_loss: -25.487396240234375 
epoch:50, d_loss:1.001341462135315, g_loss: 213.13714599609375 
epoch:100, d_loss:1.000004529953003, g_loss: 204.92572021484375 
epoch:150, d_loss:1.0000014305114746, g_loss: 218.0543212890625 
epoch:200, d_loss:1.0000026226043701, g_loss: 200.15789794921875 
epoch:250, d_loss:nan, g_loss: nan 
epoch:300, d_loss:nan, g_loss: nan 
epoch:350, d_loss:nan, g_loss: nan 
epoch:400, d_loss:nan, g_loss: nan 
epoch:450, d_loss:nan, g_loss: nan 
epoch:500, d_loss:nan, g_loss: nan 
epoch:550, d_loss:nan, g_loss: nan 
epoch:600, d_loss:nan, g_loss: nan 
epoch:650, d_loss:nan, g_loss: nan 
epoch:700, d_loss:nan, g_loss: nan 
epoch:750, d_loss:nan, g_loss: nan 
epoch:800, d_loss:nan, g_loss: nan 
epoch:850, d_loss:nan, g_loss: nan 
epoch:900, d_loss:nan, g_loss: nan 
epoch:950, d_loss:nan, g_loss: nan 
epoch:1000, d_loss:nan, g_loss: nan 
epoch:1050, d_loss:nan, g_loss: nan 
epoch:1100, d_loss:nan, g_loss: nan 
epoch:1150, d_loss:nan, g

KeyboardInterrupt: 